# Tent product review data¶

#### In this time, 
#### 1. eleminate the rating values of 3, which are meant to be neutral, to make a more sensitive model.
#### 2. save tokenizer object

In [ ]:
# ! pip install konlpy

In [2]:
import pandas as pd
import numpy as np
import pickle
import json
import matplotlib.pyplot as plt
import re
import io
import urllib.request
from konlpy.tag import Okt
from konlpy.tag import Mecab
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# 0. Load data

In [6]:
total_data = pd.read_csv('./total_data_without_val_3.csv')

In [4]:
total_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 264025 entries, 0 to 264024
Data columns (total 2 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   ratings  264025 non-null  int64 
 1   reviews  264022 non-null  object
dtypes: int64(1), object(1)
memory usage: 4.0+ MB


# 0. Preprocessing

## clean duplicated values

In [7]:
total_data['reviews'].nunique()

263576

In [8]:
total_data.drop_duplicates(subset=['reviews'], inplace=True)

## Clean null values

In [9]:
print(total_data.isnull().values.any())

True


In [10]:
total_data = total_data.dropna(how = 'any')

In [11]:
print(total_data.isnull().values.any())

False


In [12]:
total_data.reset_index(drop=True, inplace=True)

In [13]:
total_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 263576 entries, 0 to 263575
Data columns (total 2 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   ratings  263576 non-null  int64 
 1   reviews  263576 non-null  object
dtypes: int64(1), object(1)
memory usage: 4.0+ MB


## Clean unnecessary values

In [14]:
total_data['reviews'] = total_data['reviews'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
total_data['reviews'] = total_data['reviews'].str.replace('^ +', "")
total_data['reviews'].replace('', np.nan, inplace=True)
total_data = total_data.dropna(how='any')

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  """Entry point for launching an IPython kernel.
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  


In [15]:
total_data

,ratings,reviews
0,5,배공빠르고 굿
1,2,택배가 엉망이네용 저희집 밑에층에 말도없이 놔두고가고
2,5,아주좋아요 바지 정말 좋아서개 더 구매했어요 이가격에 대박입니다 바느질이 조금 엉성...
3,2,선물용으로 빨리 받아서 전달했어야 하는 상품이었는데 머그컵만 와서 당황했습니다 전화...
4,5,민트색상 예뻐요 옆 손잡이는 거는 용도로도 사용되네요 ㅎㅎ
...,...,...
263571,4,생각한것 보다 쫌 좁은느낌 그래도 렌턴걸이도 있고 다용도로 잘 쓸거 같아요 마니 파세용
263572,5,오랜 고심끝에 가성비 좋고 여러모로 장점이 많은듯 하여 솔캠 쉘터용으로 구매합니다 ...
263573,1,배송 겁나 느림 추석 전 주에 주문했는데 추석 끝나고 옴 메쉬 상태 찌그러진 곳 많...
263574,4,잘받았습니다 아직 사용전이지만 조만간 주말에 캠핑가려고 구매했너요


# 0. Split Train/Test data set

In [16]:
print(len(total_data[total_data['ratings'] == 1]))
print(len(total_data[total_data['ratings'] == 2]))
print(len(total_data[total_data['ratings'] == 4]))
print(len(total_data[total_data['ratings'] == 5]))

"""
36466
64312
29562
133178
"""

36466
64312
29562
133178


'\n36466\n64312\n29562\n133178\n'

## Labeling rating values (4 and 5 -> 1 / 1 and 2 -> 0)

In [17]:
total_data['label'] = np.select([total_data.ratings > 3], [1], default=0)
total_data[:5]

,ratings,reviews,label
0,5,배공빠르고 굿,1
1,2,택배가 엉망이네용 저희집 밑에층에 말도없이 놔두고가고,0
2,5,아주좋아요 바지 정말 좋아서개 더 구매했어요 이가격에 대박입니다 바느질이 조금 엉성...,1
3,2,선물용으로 빨리 받아서 전달했어야 하는 상품이었는데 머그컵만 와서 당황했습니다 전화...,0
4,5,민트색상 예뻐요 옆 손잡이는 거는 용도로도 사용되네요 ㅎㅎ,1


## Split train and test data as a ratio of 3:1

In [18]:
train_data, test_data = train_test_split(total_data, test_size = 0.25, random_state = 42)
print('훈련용 리뷰의 개수 :', len(train_data))
print('테스트용 리뷰의 개수 :', len(test_data))

"""
훈련용 리뷰의 개수 : 197638
테스트용 리뷰의 개수 : 65880
"""

훈련용 리뷰의 개수 : 197638
테스트용 리뷰의 개수 : 65880


'\n훈련용 리뷰의 개수 : 197638\n테스트용 리뷰의 개수 : 65880\n'

In [19]:
print(train_data.groupby('label').size().reset_index(name = 'count'))

"""
   label   count
0      0   75604
1      1  122034
"""

   label   count
0      0   75604
1      1  122034


'\n   label   count\n0      0   75604\n1      1  122034\n'

# 0. Tokenizing

In [20]:
okt = Okt()
stopwords = ['도', '는', '다', '의', '가', '이', '은', '한', '에', '하', '고', '을', '를', '인', '듯', '과', '와', '네', '들', '듯', '지', '임', '게', '좀', '으로', '하다', '걍', '네요', '잘']

In [21]:
train_data['tokenized'] = train_data['reviews'].apply(okt.morphs)
train_data['tokenized'] = train_data['tokenized'].apply(lambda x: [item for item in x if item not in stopwords])

In [23]:
train_data.to_csv('./train_data_without_val_3.csv', index=False)

In [24]:
test_data['tokenized'] = test_data['reviews'].apply(okt.morphs)
test_data['tokenized'] = test_data['tokenized'].apply(lambda x: [item for item in x if item not in stopwords])

In [25]:
test_data.to_csv('./test_data_without_val_3.csv', index=False)

##### --------------------------------------------------------------------------------------------------------------------------------------------------------------------------

# 1.Load Data

In [ ]:
#train_data = pd.read_csv('./train_data_without_val_3.csv')

In [ ]:
#test_data = pd.read_csv('./test_data_without_val_3.csv')

## split dataset for modeling

In [26]:
X_train = train_data['tokenized'].values
y_train = train_data['label'].values
X_test= test_data['tokenized'].values
y_test = test_data['label'].values

# 2. Encoding

In [27]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)

## eliminate some data appear less than 1 time

In [28]:
threshold = 2
total_cnt = len(tokenizer.word_index) # 단어의 수
rare_cnt = 0 # 등장 빈도수가 threshold보다 작은 단어의 개수를 카운트
total_freq = 0 # 훈련 데이터의 전체 단어 빈도수 총 합
rare_freq = 0 # 등장 빈도수가 threshold보다 작은 단어의 등장 빈도수의 총 합

# 단어와 빈도수의 쌍(pair)을 key와 value로 받는다.
for key, value in tokenizer.word_counts.items():
    total_freq = total_freq + value

    # 단어의 등장 빈도수가 threshold보다 작으면
    if(value < threshold):
        rare_cnt = rare_cnt + 1
        rare_freq = rare_freq + value

print('단어 집합(vocabulary)의 크기 :',total_cnt)
print('등장 빈도가 %s번 이하인 희귀 단어의 수: %s'%(threshold - 1, rare_cnt))
print("단어 집합에서 희귀 단어의 비율:", (rare_cnt / total_cnt)*100)
print("전체 등장 빈도에서 희귀 단어 등장 빈도 비율:", (rare_freq / total_freq)*100)

단어 집합(vocabulary)의 크기 : 109621
등장 빈도가 1번 이하인 희귀 단어의 수: 60146
단어 집합에서 희귀 단어의 비율: 54.86722434570018
전체 등장 빈도에서 희귀 단어 등장 빈도 비율: 2.326239796159109


In [29]:
# 전체 단어 개수 중 빈도수 2이하인 단어 개수는 제거.
# 0번 패딩 토큰과 1번 OOV 토큰을 고려하여 +2

vocab_size = total_cnt - rare_cnt + 2
print('단어 집합의 크기 :',vocab_size)

단어 집합의 크기 : 49477


In [30]:
# 단어 집합을 토크나이저의 인자로 넘겨주고, 텍스트 시퀀스를 정수 시퀀스로 변환. 
# 정수 인코딩 과정에서 이보다 큰 숫자가 부여된 단어들은 OOV로 변환.

tokenizer = Tokenizer(vocab_size, oov_token = 'OOV') 
tokenizer.fit_on_texts(X_train)
X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

## save tokenizer

In [31]:
# saving tokenizer

with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

## save word index 

In [32]:
word_index = tokenizer.word_index

In [33]:
json = json.dumps(word_index)
f3 = open("wordIndex.json", "w")
f3.write(json)
f3.close()

In [34]:
# X_train과 X_test에 대해서 상위 3개의 샘플만 출력

print(X_train[:3])
print(X_test[:3])

[[4277, 28669, 180, 317, 4634, 622, 56, 6111, 777, 1790, 816, 100, 1218, 14, 20, 324, 648, 355, 7, 19, 15212, 1111, 3077, 28670, 290, 132, 1, 2049, 9, 354], [3, 36, 1825, 17], [15, 117, 96, 470, 26, 3720]]
[[3, 1607, 798, 39, 102, 494, 456, 63, 1, 6539, 1], [33579, 1560, 2131, 368, 13433, 2132, 207, 211, 12175, 1, 214, 32, 6548, 24, 4572, 190, 1653, 83, 2, 2574, 1363, 550, 7, 1, 201], [4, 4, 316, 594, 4]]


# 3. Padding

In [35]:
def below_threshold_len(max_len, nested_list):
  count = 0
  for sentence in nested_list:
    if(len(sentence) <= max_len):
        count = count + 1
  print('전체 샘플 중 길이가 %s 이하인 샘플의 비율: %s'%(max_len, (count / len(nested_list))*100))

In [36]:
max_len = 80
below_threshold_len(max_len, X_train)

# => 훈련용 리뷰의 99.99%가 80이하의 길이를 가짐

전체 샘플 중 길이가 80 이하인 샘플의 비율: 99.62608405266194


In [37]:
# 훈련용 리뷰를 길이 80으로 패딩

X_train = pad_sequences(X_train, maxlen=max_len)
X_test = pad_sequences(X_test, maxlen=max_len)

In [38]:
print(X_train[:3])

[[    0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0  4277 28669   180   317  4634   622    56  6111   777  1790
    816   100  1218    14    20   324   648   355     7    19 15212  1111
   3077 28670   290   132     1  2049     9   354]
 [    0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     3    36  1825    17]
 [    0   

In [39]:
labels = np.asarray(X_test)
print('데이터 텐서의 크기:', X_train.shape)
print('레이블 텐서의 크기:', labels.shape)

데이터 텐서의 크기: (197638, 80)
레이블 텐서의 크기: (65880, 80)


# 8. Model GRU

In [40]:
import tensorflow
from tensorflow.keras.layers import Embedding, Dense, GRU
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [41]:
print(tensorflow.test.gpu_device_name())

/device:GPU:0


2022-06-21 02:58:20.266148: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-06-21 02:58:20.991061: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-21 02:58:21.002303: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-21 02:58:21.002911: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA 

## training

In [42]:
embedding_dim = 100
hidden_units = 128

model = Sequential()
model.add(Embedding(vocab_size, embedding_dim))
model.add(GRU(hidden_units))
model.add(Dense(1, activation='sigmoid'))

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=4)
mc = ModelCheckpoint('gru_best_model.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)

model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
history = model.fit(X_train, y_train, epochs=15, callbacks=[es, mc], batch_size=64, validation_split=0.2)

2022-06-21 02:58:28.974349: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-21 02:58:28.975109: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-21 02:58:28.975736: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-21 02:58:28.976677: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-21 02:58:28.977393: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from S

Epoch 1/15


2022-06-21 02:58:33.107999: I tensorflow/stream_executor/cuda/cuda_dnn.cc:368] Loaded cuDNN version 8200


2470/2471 [============================>.] - ETA: 0s - loss: 0.2393 - acc: 0.9084
Epoch 1: val_acc improved from -inf to 0.92297, saving model to gru_best_model.h5
2471/2471 [==============================] - 26s 8ms/step - loss: 0.2393 - acc: 0.9084 - val_loss: 0.2049 - val_acc: 0.9230
Epoch 2/15
2466/2471 [============================>.] - ETA: 0s - loss: 0.1919 - acc: 0.9311
Epoch 2: val_acc improved from 0.92297 to 0.92805, saving model to gru_best_model.h5
2471/2471 [==============================] - 18s 7ms/step - loss: 0.1919 - acc: 0.9312 - val_loss: 0.1943 - val_acc: 0.9281
Epoch 3/15
2467/2471 [============================>.] - ETA: 0s - loss: 0.1747 - acc: 0.9389
Epoch 3: val_acc improved from 0.92805 to 0.92863, saving model to gru_best_model.h5
2471/2471 [==============================] - 18s 7ms/step - loss: 0.1748 - acc: 0.9388 - val_loss: 0.1938 - val_acc: 0.9286
Epoch 4/15
2466/2471 [============================>.] - ETA: 0s - loss: 0.1620 - acc: 0.9442
Epoch 4: val_ac

In [43]:
gru_loaded_model = load_model('gru_best_model.h5')
print("\n 테스트 정확도: %.4f" % (gru_loaded_model.evaluate(X_test, y_test)[1]))

2059/2059 [==============================] - 7s 3ms/step - loss: 0.1983 - acc: 0.9292

 테스트 정확도: 0.9292


## predict with the trained GRU model

In [44]:
okt = Okt()
stopwords = ['도', '는', '다', '의', '가', '이', '은', '한', '에', '하', '고', '을', '를', '인', '듯', '과', '와', '네', '들', '듯', '지', '임', '게', '좀', '으로', '하다', '걍', '네요', '잘']

In [45]:
def gru_predict(new_sentence):
    new_sentence = re.sub(r'[^ㄱ-ㅎㅏ-ㅣ가-힣 ]','', new_sentence)
    new_sentence = okt.morphs(new_sentence)
    new_sentence = [word for word in new_sentence if not word in stopwords]
    encoded = tokenizer.texts_to_sequences([new_sentence])
    pad_new = pad_sequences(encoded, maxlen = max_len)

    score = float(gru_loaded_model.predict(pad_new))
    if(score > 0.5):
        print("{:.2f}% 확률로 긍정 리뷰입니다.".format(score * 100))
    else:
        print("{:.2f}% 확률로 부정 리뷰입니다.".format((1 - score) * 100))

In [46]:
gru_predict('각 관절마다 원터치 폴딩 방식의 오토형 쉘터이다보니 역시나 피칭과 정리가 간편하여 요즘 같이 더워지는 시기에 사용하기 너무 안성맞춤 입니다.')

99.89% 확률로 긍정 리뷰입니다.


In [47]:
gru_predict('익숙하지 않아서 인지 조금 힘드네요~ㅜ 도킹을 안해서인지 팩을 가이라인까지 10개나 박았는데 다른텐트에 비해 너무 펄럭입니다. 바람4인데 뒤집어 질뻔 했어요~ 버티겠지요? 카프리콘도 주문했는데 살짤 고민되네요...')

89.42% 확률로 부정 리뷰입니다.


In [48]:
gru_predict('답답하구 이너안으로 전기선 넣는곳도 없구 비효율적입니다. 비추에요')

93.03% 확률로 부정 리뷰입니다.


# 9. Save the trained GRU model

In [49]:
from tensorflow import keras
import os

In [50]:
# save the model as  SavedModel format
SAVED_MODEL_PATH = 'gru_review_saved_model_0621'
#make_directory(SAVED_MODEL_PATH)
MODEL_DIR = SAVED_MODEL_PATH

version = 1
export_path = os.path.join(MODEL_DIR, str(version))
print('export_path = {}\n'.format(export_path))


tensorflow.keras.models.save_model(
  model,
  export_path,
  overwrite=True,
  include_optimizer=True,
  save_format=None,
  signatures=None,
  options=None
)

export_path = gru_review_saved_model_0621/1



2022-06-21 03:02:11.524513: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: gru_review_saved_model_0621/1/assets


INFO:tensorflow:Assets written to: gru_review_saved_model_0621/1/assets
